In [3]:
import ctypes

try:
    pco_dll_file = 'SC2_Cam.dll'
    pco_dll = ctypes.WinDLL(pco_dll_file)
except:
    #quickfix
    pco_dll_file = 'C:\\Program Files (x86)\\pco\\pco.sdk\\bin64\\SC2_Cam.dll' # Quickfix...
    pco_dll = ctypes.WinDLL(pco_dll_file)
pco_dll

 # opencamera is the instance of OpenCamera method in DLL
opencamera = pco_dll.PCO_OpenCamera
# PCO_OpenCamera(HANDLE *hCam, int board_num), return int
opencamera.argtypes = (ctypes.POINTER(ctypes.c_int), ctypes.c_int)
opencamera.restype = ctypes.c_int
hCam = ctypes.c_int()
# return 0 if success, <0 if error
camId = int(0)
ret = opencamera(hCam, camId)
if ret < 0:
    print('Error connecting PCO camera: id = {0}'.format(camId))


In [3]:
PCOCam(0)

PCO - Binning: [2, 2]
PCO - Exposure: 100 ms
PCO - size: 696 x 520
True


<PCOCam(, initial)>

In [2]:
from labcams.cams import *
import ctypes
class PCOCam(GenericCam):
    time_modes = {0:"ns", 1: "us", 2: "ms"}
    armed = False
    def __init__(self, camId = None, outQ = None,
                 frameRate = 30.,
                 binning = 2,
                 dtype = np.uint8,
                 triggered = Event(),
                 dllpath = 'C:\\Program Files (x86)\\pco\\pco.sdk\\bin64\\SC2_Cam.dll',
                 **kwargs):
        super(PCOCam,self).__init__()
        self._dll = ctypes.WinDLL(dllpath)
        self.h = None
        self.w = None
        self.closeEvent = Event()
        self.startTrigger = Event()
        self.stopTrigger = Event()
        self.saving = Event()
        self.nframes = Value('i',0)
        if camId is None:
            display('Need to supply a camera ID.')
        self.camId = camId
        self.frameRate = frameRate
        self.queue = outQ
        self.dtype = dtype
        ret = self.camopen(self.camId)
        assert ret == 0, "PCO: Could not open camera {0}".format(camId)
        ret = self.set_binning(binning,binning)
        print('PCO - Binning: {0}'.format(ret))
        ret = self.set_exposure_time()
        print('PCO - Exposure: {0} {1}'.format(*ret))
        self.arm()
        print('PCO - size: {0} x {1}'.format(self.h,self.w))
        self.disarm()
        self.camclose()
        
#         ret_val, frame = cam.read()
#         frame = frame[:,:,0]
#         self.h = frame.shape[0]
#         self.w = frame.shape[1]
#         self.nchannels = 1 #frame.shape[2]
#         self.initVariables()
#         buf = np.frombuffer(self.frame.get_obj(),
#                             dtype = np.uint8).reshape([self.h,
#                                                        self.w])
        
#         buf[:,:] = frame[:,:]
#         display("Got info from camera (name: {0})".format(
#             'openCV'))
#         cam.release()
#         self.cameraReady = Event()
#         self.triggered = triggered
#         if self.triggered.is_set():
#             display('[OpenCV {0}] Triggered mode ON.'.format(self.camId))
#             self.triggerSource = triggerSource
    
    def camopen(self,camid,reset = True):
        '''Open PCO camera'''
        opencamera = self._dll.PCO_OpenCamera
        # PCO_OpenCamera(HANDLE *hCam, int board_num), return int
        opencamera.argtypes = (ctypes.POINTER(ctypes.c_int), ctypes.c_int)
        opencamera.restype = ctypes.c_int
        self.hCam = ctypes.c_int()
        ret = opencamera(self.hCam, camid)
        if ret == 0 and reset:
            self._dll.PCO_ResetSettingsToDefault(self.hCam)
        return ret
    
    def camclose(self):
        ''' Close PCO camera'''
        return self._dll.PCO_CloseCamera(self.hCam)
    
    def set_binning(self, h_bin, v_bin):
        """
        binning allows for Binning pixels in h_bin x v_bin
        Allowed values in {1,2,4,8,16,32}
        :param h_bin: binning in horizontal direction
        :param v_bin:
        :return: None
        """
        allowed = [1, 2, 4]
        wBinHorz = ctypes.c_uint16(int(h_bin))
        wBinVert = ctypes.c_uint16(int(v_bin))
        if (h_bin in allowed) and (v_bin in allowed):
            self._dll.PCO_SetBinning(self.hCam, wBinHorz, wBinVert)
            self._dll.PCO_GetBinning(self.hCam, ctypes.byref(wBinHorz),
                                          ctypes.byref(wBinVert))
            return [wBinHorz.value, wBinVert.value]
        else:
            raise ValueError("Not allowed binning value pair " + str(h_bin)
                              + "x" + str(v_bin))
            return None
    
    def set_exposure_time(self, exp_time=100, base_exposure=2):
        """
        Sets delay and exposure time allowing to choose a base for each parameter
        0x0000 timebase=[ns]=[10^-9 seconds]
        0x0001 timebase=[us]=[10^-6 seconds]
        0x0002 timebase=[ms]=[10^-3 seconds]
        Note: Does not require armed camera to set exp time
        :param exp_time: Exposure time (integer < 1000)
        :param base_exposure: Base 10 order for exposure time in seconds-> ns/us/ms
        :param verbose: True if process should be printed
        :return: None
        """
        # check for allowed values
        if not(base_exposure in [1, 2]):
            raise UserWarning("PCO - Not accepted time base mode (has to be 1 or 2).")

        # pass values to ctypes variables
        dwDelay = ctypes.c_uint32(0)
        dwExposure = ctypes.c_uint32(int(exp_time))
        wTimeBaseDelay = ctypes.c_uint16(0)
        wTimeBaseExposure = ctypes.c_uint16(int(base_exposure))

        # set exposure time and delay time
        self._dll.PCO_SetDelayExposureTime(self.hCam,
                                           dwDelay, dwExposure,
                                           wTimeBaseDelay, wTimeBaseExposure)
        self._dll.PCO_GetDelayExposureTime(self.hCam, ctypes.byref(dwDelay),
                                           ctypes.byref(dwExposure),
                                           ctypes.byref(wTimeBaseDelay),
                                           ctypes.byref(wTimeBaseExposure))

        return [dwExposure.value, self.time_modes[wTimeBaseExposure.value]]

    def get_exposure_time(self):
        """
        Get exposure time of the camera.
        :return: exposure time, units
        """
        # pass values to ctypes variables
        dwDelay = ctypes.c_uint32(0)
        dwExposure = ctypes.c_uint32(0)
        wTimeBaseDelay = ctypes.c_uint16(0)
        wTimeBaseExposure = ctypes.c_uint16(0)

        # get exposure time
        self._dll.PCO_GetDelayExposureTime(self.hCam, ctypes.byref(dwDelay),
                                           ctypes.byref(dwExposure),
                                           ctypes.byref(wTimeBaseDelay),
                                           ctypes.byref(wTimeBaseExposure))

        return [dwExposure.value, self.time_modes[wTimeBaseExposure.value]]

    def arm(self):
        """
        Arms camera and allocates buffers for image recording
        :return:
        """
        if self.armed:
            
            raise UserWarning("PCO - Camera already armed?")

        # Arm camera
        self._dll.PCO_ArmCamera(self.hCam)
        # Get the actual image resolution-needed for buffers
        self.wXResAct, self.wYResAct, wXResMax, wYResMax = (
            ctypes.c_uint16(), ctypes.c_uint16(), ctypes.c_uint16(),
            ctypes.c_uint16())
        self._dll.PCO_GetSizes(self.hCam, ctypes.byref(self.wXResAct),
                               ctypes.byref(self.wYResAct), ctypes.byref(wXResMax),
                               ctypes.byref(wYResMax))
        self.h,self.w = [self.wXResAct.value,
                         self.wYResAct.value]
        self.armed = True
        self.allocate_buffers()
        return self.armed
    
    def disarm(self):
        """
        Disarm camera, free allocated buffers and set
        recording to 0
        :return:
        """
        # set recording state to 0
        wRecState = ctypes.c_uint16(0)
        self._dll.PCO_SetRecordingState(self.hCam, wRecState)
        # free all allocated buffers
        self._dll.PCO_RemoveBuffer(self.hCam)
        for buf in self.buffer_numbers:
            self._dll.PCO_FreeBuffer(self.hCam, buf)
        self.buffer_numbers, self.buffer_pointers, self.buffer_events = (
            [], [], [])
        self.armed = False
    
    def allocate_buffers(self, num_buffers=2):
        """
        Allocate buffers for image grabbing
        :param num_buffers:
        :return:
        """
        dwSize = ctypes.c_uint32(self.wXResAct.value*self.wYResAct.value*2)  # 2 bytes per pixel
        # set buffer variable to []
        self.buffer_numbers, self.buffer_pointers, self.buffer_events = ([], [], [])
        # now set buffer variables to correct value and pass them to the API
        for i in range(num_buffers):
            self.buffer_numbers.append(ctypes.c_int16(-1))
            self.buffer_pointers.append(ctypes.c_void_p(0))
            self.buffer_events.append(ctypes.c_void_p(0))
            self._dll.PCO_AllocateBuffer(self.hCam, ctypes.byref(self.buffer_numbers[i]),
                                         dwSize, ctypes.byref(self.buffer_pointers[i]),
                                         ctypes.byref(self.buffer_events[i]))

        # Tell camera link what actual resolution to expect
        self._dll.PCO_CamLinkSetImageParameters(self.hCam, self.wXResAct, self.wYResAct)

    
    def run(self):
        buf = np.frombuffer(self.frame.get_obj(),
                            dtype = np.uint8).reshape([self.h,self.w])
        self.closeEvent.clear()
        
        while not self.closeEvent.is_set():
            self.nframes.value = 0
            cam = cv2.VideoCapture(self.camId)
            self.cameraReady.set()
            self.nframes.value = 0
            # Wait for trigger
            display('OpenCV camera [{0}] waiting for software trigger.'.format(self.camId))
            while not self.startTrigger.is_set():
                # limits resolution to 1 ms 
                time.sleep(0.001)
                if self.closeEvent.is_set():
                    break
            if self.closeEvent.is_set():
                break
            display('OpenCV [{0}] - Started acquisition.'.format(self.camId))
            self.cameraReady.clear()
            while not self.stopTrigger.is_set():
                timestamp = 0
                frameID = self.nframes.value
                ret_val, frame = cam.read()
                frame = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
                self.nframes.value += 1
                if self.saving.is_set():
                    if not frameID in lastframeid :
                        self.queue.put((frame.copy(),(frameID,timestamp)))
                        lastframeid[ibuf] = frameID
                buf[:,:] = frame[:,:]
            cam.release()
            display('OpenCV [{0}] - Stopped acquisition.'.format(self.camId))
            self.saving.clear()
            self.startTrigger.clear()
            self.stopTrigger.clear()
            display('OpenCV {0} - Close event: {1}'.format(self.camId,
                                                           self.closeEvent.is_set()))


In [6]:
 def pco_get_exposure_time(hCam):
    """
    Get exposure time of the camera.
    :return: exposure time, units
    """
    # pass values to ctypes variables
    dwDelay = ctypes.c_uint32(0)
    dwExposure = ctypes.c_uint32(0)
    wTimeBaseDelay = ctypes.c_uint16(0)
    wTimeBaseExposure = ctypes.c_uint16(0)

    # get exposure time
    pco_dll.PCO_GetDelayExposureTime(hCam, ctypes.byref(dwDelay),
                                            ctypes.byref(dwExposure),
                                            ctypes.byref(wTimeBaseDelay),
                                            ctypes.byref(wTimeBaseExposure))

    return [dwExposure.value, self.time_modes[wTimeBaseExposure.value]]
pco_get_exposure_time(hCam)

IndentationError: unindent does not match any outer indentation level (<ipython-input-6-65649a1513e7>, line 18)